# Authors

*The work can be done in groups of up to 3 students. Please complete the following fields with your group number and list your names along with ISU ID numbers.*

---
> **Тех.Зр R23 1.1** *[R3243] Technical Vision*
> 1. Dinislam Sayfullin, 409502
> 2. Roman Bakhtairov, 408233
---

The task and guidelines were prepared by Andrei Zhdanov And Sergei Shavetov from ITMO University in 2024.

# Practical Assignment No 3. Filtering and Edges Detection

Studying of the filtering images basic methods and edges detection.

## Task 1. Noises types

*Select an arbitrary image. Get images distorted by various noises using the imnoise() function with parameters other than the default values.*

### 1.1 Preparation

First, we need to add some imports for OpenCV to work. Python implementation of OpenCV is in `cv2` library, but for convenience it will be imported as `cv`.

In [35]:
import cv2
import cv2 as cv
import numpy
import numpy as np
import skimage
import sys

Also we will import `ShowImages()` function from the first practical assignment to use it here. It is placed in `lab_utils`.

In [36]:
from pa_utils import ShowImages

### 1.2 Read and display an image

Now let's open our image which we will use during the current task. We will open it both in GBR and convert to grayscale. We will use previously introducted functions for image display.

In [ ]:
# Read an image from file in BGR
fn = "images/lena_color.png"
I = cv.imread(fn, cv.IMREAD_COLOR)
if not isinstance(I, np.ndarray) or I.data == None:
  print("Error reading file \"{}\"".format(fn))
  exit()

# Convert BGR to grayscale
I = cv.cvtColor(I, cv.COLOR_BGR2GRAY)

# Display it
ShowImages([("Source image", I)])

### 1.3 Noises

Digital images obtained by various optoelectronic devices can contain various distortions caused by various kinds of interference, which are commonly called *noise*. The noise in the image makes it difficult to process it automatically. The noises have a different nature, and for its successful suppression it is necessary to determine an adequate mathematical model. Let's consider the most common noise models. When using OpenCV noise have to be added manually, however the `skimage` Python toolbox proveides `skimage.util.random_noise()` function wich allows adding noise. We will try both ways.

#### 1.3.1 Impulse noise

With impulse noise, the signal is distorted by spikes with very large negative or positive values of short duration and can arise, for example, due to decoding errors. This noise results in white "*salt*" or black "*pepper*" dots in the image, which is why it is often called *dot* noise. To describe it, one should take into account the fact that the appearance of a noise spike in each pixel $I(x,y)$ does not depend on the quality of the original image or on the presence of noise at other points and has the probability of $p$, and the value of the pixel intensity $I(x,y)$ will be changed to value $d\in[0,255]$:

$$
	I_{noisy}(x,y) =
	\begin{cases}
		d      & ~\text{with probability}~p    \\
		I(x,y) & ~\text{with probability}~(1-p)
	\end{cases}
$$

where $I(x,y)$ is the pixel intensity of the original image, $I_{noisy}$ is the noisy image, and $p$ is the nose ratio. If $d=0$ then the noise "*pepper*" type, if $d=255$ then noise is "*salt*".


Let's implement the impulse noise. As for the *salt* type noise we have to add the maximum value, we will consider that only floating point images with $1.0$ maximum value, or `uint8` ones with $255$ maximum value are supported. The noise is controlled by two values: the noise probability (`p`) and salt versus pepper ratio (`s_vs_p`).

In [ ]:
## Add the salt and pepper noise to an image
## @param[in] image An image
## @param[in] p Probability of noise
## @param[in] s_vs_p Salt vs pepper distribution
## @return An image with noise
def SaltAndPepper(I, p = 0.05, s_vs_p = 0.5):
  # Select the salt value
  if I.dtype == np.uint8:
    salt = 255
  elif I.dtype == np.float32:
    salt = 1.0
  else:
    print("Unsopported image type")
    return None

  # Create an image
  Inoise = np.copy(I)
  # Get random value for each image poxel
  rng = np.random.default_rng()
  vals = rng.random(Inoise.shape)
  # Add salt
  Inoise[vals < p * s_vs_p] = salt
  # Add pepper
  Inoise[np.logical_and(vals >= p * s_vs_p, vals < p)] = 0

  return Inoise

Alternatively, the `skimage.util.random_noise(I, 's&p')` function can be used. Its optional `amount` parameter can be used to modify the "*salt*" and "*pepper*" probabilities. The next optional `salt_vs_pepper`  can be used to set the relative probabilities of "*salt*" and "*pepper*" noises. Only "*salt*" noise can be generated by `skimage.util.random_noise(I, 'salt')` function, while only "*pepper*" can be generated by `skimage.util.random_noise(I, 'pepper')` function. It should also be noted that these methods converts an image to a floating point value image, so additional conversion is requred.

Let's run both this methods and compare results.

In [ ]:
# Own implementation
Isnp = SaltAndPepper(I, 0.1, 0.5)
# Skimage
Isnp2 = (skimage.util.random_noise(I, "s&p", amount = 0.1, salt_vs_pepper = 0.5) * 255).astype(np.uint8)
# Show both
ShowImages([("Salt & Pepper", Isnp),
            ("SKImage Salt & Pepper", Isnp2)])

We may also approximately calculate the resulting *salt* and *pepper* ratios with simple math transformations. Let's do it.

In [ ]:
diff = I.astype(np.int32) - Isnp
print("Own implementation salt ratio is {:.2f}%, pepper ratio is {:.2f}%".format(
      100 * np.count_nonzero(diff < 0) / I.shape[0] / I.shape[1],
      100 * np.count_nonzero(diff > 0) / I.shape[0] / I.shape[1]))
diff2 = I.astype(np.int32) - Isnp2
# As skimage work in floating point values with conversion,
# so we have to take this into an account and add some epsilon value
print("Skimage implemetation salt ratio is {:.2f}%, pepper ratio is {:.2f}%".format(
      100 * np.count_nonzero(diff2 < -1) / I.shape[0] / I.shape[1],
      100 * np.count_nonzero(diff2 > 1) / I.shape[0] / I.shape[1]))

Own implementation salt ratio is 4.99%, pepper ratio is 5.04%
Skimage implemetation salt ratio is 5.02%, pepper ratio is 4.97%


#### 1.3.2 Additive noise

Additive noise is described by the following expression:

$$I_{noisy}(x,y) = I(x,y) + \eta(x,y)$$

where $I_{noisy}$ is the noisy image, $I$ is the original image, and $\eta$ is the signal-independent additive noise with Gaussian or any other probability density function.

We will see an example of an additive noise a bit later in a Gaussian noise section.

#### 1.3.3 Multiplicative Noise

Multiplicative noise is described by the following expression:

$$I_{noisy}(x,y) = I(x,y) \cdot \eta(x,y)$$

wheew $I_{noisy}$ is the noisy image, $I$ is the original image, and $\eta$ is the signal independent multiplicative noise that multiplies the recorded signal. Examples include film grain, ultrasound images, etc. A special case of multiplicative noise is *speckle* noise. This noise appears in images captured by coherent imaging devices such as medical scanners or radars. In such images, one can clearly observe light spots, speckles, which are separated by dark areas of the image.


Let's implement the multiplicative noise with normal distributiosn which is called the *speckle* noise.

In [ ]:
## Add the speckle noise to an image
## @param[in] image An image
## @param[in] var Random number variance
## @return An image with noise
def Speckle(I, var = 0.05):
  if I.dtype != np.uint8 and I.dtype != np.float32:
      print("Unsopported image values type.")
      return None

  # Generate a random number with normal distribution for each image pixel
  rng = np.random.default_rng()
  gauss = rng.normal(0, var ** 0.5, I.shape)

  # Add a multiple of this distribution to our image
  # The image values type should be taken into an account here
  # as we should work with floating point values
  if I.dtype == np.uint8:
    out = (I.astype(np.float32) * (gauss + 1)).clip(0, 255).astype(np.uint8)
  else:
    out = I * (gauss + 1)

  return out

This noise can also be applied to an image by using `skimage.util.random_noise(I, 'speckle')`. Optional `mean` and `var` parameters defines the mean and variance for the normal distribution used to generate noise.

Let's run both this methods and compare results.

In [ ]:
# Own implementation
Isp = Speckle(I, 0.05)
# Skimage
Isp2 = (skimage.util.random_noise(I, "speckle", mean = 0, var = 0.05) * 255).astype(np.uint8)
# Show both
ShowImages([("Speckle", Isp),
            ("SKImage Speckle", Isp2)])

#### 1.3.4 Gaussian (Normal) Noise

Gaussian noise in the additive noise on the image that can occur due to a lack of scene illumination, high temperature, etc. The noise model is widely used in low-pass filtering of images. The probability density distribution function $p(z)$ of the random variable $z$ is described by the following expression:

$$p(z) = \dfrac{1}{\sigma \sqrt{2\pi}}e^{\frac{-(z-\mu)^2}{2\sigma^2}}$$

where $z$ is the the intensity of the image (for example, for a grayscale image $z\in[0,255]$), $\mu$ is the mean (mathematical expectation) of a random variable $z$, $\sigma$ is the standard deviation, variance $\sigma^2$ determines the power of the introduced noise. Approximately $67%$ of the random variable $z$ values is concentrated in the range $\left[(\mu-\sigma),(\mu+\sigma)\right]$ and about $96%$ in the range $\left[(\mu-2\sigma),(\mu+2\sigma)\right]$.


As we know that this is an additive noise and it uses the normal distribution, we can easily implement it.

In [ ]:
## Add the Gaussian noise to an image
## @param[in] image An image
## @param[in] mean Mean value
## @param[in] var Variance
## @return An image with noise
def Gaussian(I, mean = 0, var = 0.01):
  if I.dtype != np.uint8 and I.dtype != np.float32:
    print("Unsopported image values type.")
    return None

  rng = np.random.default_rng()
  gauss = rng.normal(mean, var ** 0.5, I.shape)

  if I.dtype == np.uint8:
    out = (I.astype(np.float32) + gauss * 255).clip(0, 255).astype(np.uint8)
  else:
    out = (I + gauss).astype(np.float32)

  return out

It can also be applied to an image by using `skimage.util.random_noise(I, 'gaussian')` function. Optional `mean` and `var` parameters defines the mean and variance for the normal distribution used to generate noise. The additional `skimage.util.random_noise(I, 'localvar')` function allows to specify the local random variance for each pixel of the image passed by an extra `local_vars` function parameter.

Let's run both this methods and compare results.

In [ ]:
# Own implementation
Igauss = Gaussian(I, 0, 0.05)
# Skimage
Igauss2 = (skimage.util.random_noise(I, "gaussian", mean = 0, var = 0.05) * 255).astype(np.uint8)
# Show both
ShowImages([("Gaussian", Igauss),
            ("SKImage Gaussian", Igauss2)])

#### 1.3.5 Quantization Noise

Depends on the selected quantization step and on the signal. Quantization noise can lead, for example, to the appearance of false contours around objects or to remove low-contrast details in the image. Such noise is not eliminated. Quantization noise can be approximately described by the Poisson distribution.

Since we know the distribution type, we can implement this noise.

In [ ]:
## Add the poison noise to an image
## @param[in] image An image
## @return An image with noise
def Poisson(I):
  rng = np.random.default_rng()
  # We have to calculate the number of unique values then add
  # the poisson distribution noise basing on this number to
  # simulate the quantization noise
  if I.dtype == np.uint8:
    Ifloat = I.astype(np.float32) / 255
    vals = len(np.unique(Ifloat))
    vals = 2 ** np.ceil(np.log2(vals))
    out = (255 * (rng.poisson(Ifloat * vals) / float(vals)).clip(0, 1)).astype(np.uint8)
  elif I.dtype == np.float32:
    vals = len(np.unique(I))
    vals = 2 ** np.ceil(np.log2(vals))
    out = rng.poisson(I * vals) / float(vals)
  else:
    print("Unsopported image values type.")
    return None

  return out

It can also be applied  by using `skimage.util.random_noise(I, 'poisson')` function.

Let's run both this methods and compare results.

In [ ]:
# Own implementation
Ipoisson = Poisson(I)
# Skimage
Ipoisson2 = (skimage.util.random_noise(I, "poisson") * 255).astype(np.uint8)
# Show both
ShowImages([("Poisson", Ipoisson),
            ("SKImage Poisson", Ipoisson2)])

#### 1.3.6 Noise summary

So, we learned how to add different type of noise. Let's write a copy of MATLAB's `imnoise` function.

In [ ]:
## Add noise to an image
## @param[in] image An image
## @param[in] noise_type Noise type to add
## @param[in] param1 First noise-dependent parameter
## @param[in] param2 Second noise-dependent parameter
## @return An image with noise
## @note For "salt & pepper" type, param1 is the probability, param2 is the salt vs pepper ratio.
## @note For "speckle" type, param1 is variance.
## @note For "gaussian" type, param1 is mean, param2 is variance.
def imnoise(I, noise_type, param1 = None, param2 = None):
  # Salt & pepper (param1 - probability, param2 - salt vs pepper)
  if noise_type == "salt & pepper":
    if param1 != None:
      d = param1
    else:
      d = 0.05
    if param2 != None:
      s_vs_p = param2
    else:
      s_vs_p = 0.5

    return SaltAndPepper(I, d, s_vs_p)

  # Multiplicative Noise (param1 - variance)
  if noise_type =="speckle": # Variance of multiplicative noise, specified as a numeric scalar
    if param1 != None:
      var = param1
    else:
      var = 0.05

    return Speckle(I, var)

  # Gaussian Noise (param1 - mean, param2 - variance)
  if noise_type == "gaussian":
    if param1 != None:
      mean = param1
    else:
      mean = 0
    if param2 != None:
      var = param2
    else:
      var = 0.01

    return Gaussian(I, mean, var)

  # Quantization Noise
  if noise_type == "poisson":
    return Poisson(I)

  return None


Also, let's make a more convinient array of images to test different noise filters.

In [ ]:
Itmp = I # cv.resize(I, (256, 256))
Inoise = [("Salt & Pepper", SaltAndPepper(Itmp, 0.1)),
          ("Speckle", Speckle(Itmp)),
          ("Gaussian", Gaussian(Itmp)),
          ("Poisson", Poisson(Itmp))]
ShowImages(Inoise, 0)

---
> ***Self-work***
>
> Take some arbitrary image and apply different noises to it. Show the results and store them in an array to test filters.
---

In [ ]:
# Read an image from file in BGR
fn = 'images/my_img.png'
I2 = cv.imread(fn, cv.IMREAD_COLOR)
if not isinstance(I2, np.ndarray) or I2.data == None:
  print('Error reading file \"{}\"'.format(fn))
  exit()

# Convert BGR to grayscale
I2 = cv.cvtColor(I2, cv.COLOR_BGR2GRAY)

# Display it
ShowImages([('Source image', I2)])

IsaltAndPepper = [('Salt & Pepper v1', imnoise(I2, 'salt & pepper')),
                  ('Salt & Pepper v2', imnoise(I2, 'salt & pepper', param1 = 0.2)),
                  ('Salt & Pepper v3', imnoise(I2, 'salt & pepper', param2 = 0.3))]
ShowImages(IsaltAndPepper, 3)

Ispeckle = [('Speckle v1', imnoise(I2, 'speckle')),
            ('Speckle v2', imnoise(I2, 'speckle', param1 = 0.07)),
            ('Speckle v3', imnoise(I2, 'speckle', param1 = 0.02))]
ShowImages(Ispeckle, 3)

Igaussian = [('Gaussian v1', imnoise(I2, 'gaussian')),
            ('Gaussian v2', imnoise(I2, 'gaussian', param1 = 0.1)),
            ('Gaussian v3', imnoise(I2, 'gaussian', param2 = 0.05))]
ShowImages(Igaussian, 3)

Ipoisson = [('Poisson', imnoise(I2, 'poisson'))]
ShowImages(Ipoisson, 0)

Inoise2 = [('Salt & Pepper', imnoise(I2, 'salt & pepper', param1 = 0.2)),
           ('Speckle', imnoise(I2, 'speckle', param1 = 0.07)),
           ('Gaussian', imnoise(I2, 'gaussian', param2 = 0.05)),
           ('Poisson', imnoise(I2, 'poisson'))]
ShowImages(Inoise2, 0)

## Task 2. Low-pass filtering

*Process the distorted images obtained in the previous paragraph with considered low-pass filters and a counterharmonic mean filter with different values of the parameter 𝑄.*

### 2.1 Sliding window filters

Let's consider the main methods of image filtering. It the neighboring pixels values in a certain neighborhood are taken into account to calculate th intensity value of the each pixel, then such a transformation is called $\text{local}$, and the neighborhood is called a *window*. The "window" is called *mask*, *filter*, *filter kernel*, and matrix coefficients are called *coefficients*. The center of the mask is aligned with the analyzed pixel, and the mask coefficients are multiplied by the intensities of the pixels covered by the mask. Typically, the mask has a square shape of $3\times 3$, $5\times 5 $, etc. Filtering an $M\times N$ image $I$ using a mask of $m\times n$ size is described by the formula:

$$I_{filtered}(x,y) = \sum_s \sum_t w(s,t)I(x+s, y+t)$$

where $s$ and $t$ are the coordinates of the mask elements relative to its center (in the center $s=t=0$). Such transformations are called *linear*. After calculating the new value of the pixel intensity $I_{filtered}(x,y)$ window $w$, n which the filter mask is described, it is shifted and the intensity of the next pixel is calculated. Therefore, such a transformation is called *sliding window filtering*.


When using OpenCV image filtering can be performed by calling the `dst = cv2.filter2D(src, ddest, kernel)` function. Here `src` is an input image matrix, `dst` is an output image matrix, `kernel` is filtering kernel that should be specified manually. Filtering mask can be created manually by creating new numpy matrix, for example:

```python
mask = np.float64([[1, 1, 1], [1, 1, 1], [1, 1, 1]])
```

### 2.2 Low-pass filter kernels

Low-pass spatial filters attenuate the high-pass components (areas with large intensity changes) and leave the low-pass components of the image unchanged. Used to reduce noise and remove high-pass components, which improves the accuracy of the study of the content of low-pass components. As a result of applying low-pass filters, we get a smoothed or blurred image. The main distinguishing features are:

- non-negative mask coefficients;
- the sum of all coefficients is equal to one.

Examples of low-pass filter kernels:

$$
	w_1 = \dfrac{1}{9}
	\begin{bmatrix}
		1 & 1 & 1
		\\ 1 & 1 & 1
		\\ 1 & 1 & 1
	\end{bmatrix},
	w_2 = \dfrac{1}{10}
	\begin{bmatrix}
		1 & 1 & 1
		\\ 1 & 2 & 1
		\\ 1 & 1 & 1
	\end{bmatrix}.
$$

Let's try running the low-pass filter with these kernels for all our four noise types. First we implement kernel $w_2$.

In [ ]:
w1 = np.ones((3, 3)) / 9
Iw1 = []
for img in Inoise:
  Iw1.append((img[0], cv.filter2D(img[1], -1, w1)))
ShowImages(Iw1, 0)

---
> **Self-work**
>
> Run the low-pass 2D filter with kernel $w_2$ for all noise types.
---

In [ ]:
w2 = np.ones((3, 3))
w2[1][1] = 2
print(w2)
w2 /= 10
Iw2 = []
for img in Inoise:
  Iw2.append((img[0], cv.filter2D(img[1], -1, w2)))
ShowImages(Iw2, 0)

### 2.3 Arithmetic mean filter

This filter averages the pixel intensity value over the neighborhood using a mask with the same coefficients, for example, for a mask with the size $3 \times 3$ the coefficients are $1/9$, if $5 \times 5$ --- $1/25$. Thanks to this normalization, the value of the filtering result will be reduced to the original image intensities range. Graphically, the two-dimensional function describing the filter mask looks like a parallelepiped, so the name is used in English literature *box*-filter. Arithmetic averaging is achieved using the following formula:

$$I_{filtered}(x,y) = \dfrac{1}{m \cdot n} \sum_{i=0}^m \sum_{j=0}^n I(i,j)$$

where $I_{filtered}(x,y)$ is fht the value of the filtered image pixel intensity, $I(i,j)$ is the the value of the original image in the mask pixels intensities, $m$ and $n$ are the filter mask width and height respectively. This algorithm is effective for slightly noisy images. In OpenCV it can be executed by using the function `cv2.blur(I, (3, 3))` for $3 \times 3$ filter kernel.


In [ ]:
Iblur = []
for img in Inoise:
  Iblur.append((img[0], cv.blur(img[1], (3, 3))))
ShowImages(Iblur, 0)

In [ ]:
Iblur2 = []
for img in Inoise2:
  Iblur2.append((img[0], cv.blur(img[1], (3, 3))))
ShowImages(Iblur2, 0)

### 2.4 Geometric mean filter

Geometric averaging is calculated using the formula:

$$I_{filtered}(x,y) = \left[\prod_{i=0}^{m}\prod_{j=0}^{n} I(i,j)\right]^{\dfrac{1}{m \cdot n}}$$

The effect of applying this filter is similar to the previous method, but individual objects in the original image are less distorted. The filter can be used to suppress high-pass additive noise with better statistical performance than an arithmetic averaging filter.


This filter can be easily implemented via arithmetic mean filter since $\prod_{i}x_i = e^{\sum_{i}\ln{}x_i}$, so

$$I_{filtered}(x,y) = \left[\prod_{i=0}^{m}\prod_{j=0}^{n} I(i,j)\right]^{\dfrac{1}{m \cdot n}} = \left[e^{\sum_{i=0}^{m}\sum_{j=0}^{n} \ln{I(i,j)}}\right]^{\dfrac{1}{m \cdot n}}$$

Let's implement it for a $5 \times 5$ kernel.

In [ ]:
kernel = np.ones((5, 5)) / 25
Igeom = []
for img in Inoise:
  Itmp = img[1].copy()
  if img[1].dtype == np.uint8:
    Itmp[Itmp == 0] = 1
    Ifilter = np.power(np.exp(cv.filter2D(np.log(Itmp).astype(np.float32), -1, kernel)), np.sum(kernel)).clip(0, 255).astype(np.uint8)
  else:
    Itmp[Itmp == 0] = sys.float_info.epsilon
    Ifilter = np.power(np.exp(cv.filter2D(np.log(Itmp).astype(np.float32), -1, kernel)), np.sum(kernel))
  Igeom.append((img[0], Ifilter))

ShowImages(Igeom, 0)

In [ ]:
kernel = np.ones((5, 5)) / 25
Igeom2 = []
for img in Inoise2:
  Itmp = img[1].copy()
  if img[1].dtype == np.uint8:
    Itmp[Itmp == 0] = 1
    Ifilter = np.power(np.exp(cv.filter2D(np.log(Itmp).astype(np.float32), -1, kernel)), np.sum(kernel)).clip(0, 255).astype(np.uint8)
  else:
    Itmp[Itmp == 0] = sys.float_info.epsilon
    Ifilter = np.power(np.exp(cv.filter2D(np.log(Itmp).astype(np.float32), -1, kernel)), np.sum(kernel))
  Igeom2.append((img[0], Ifilter))

ShowImages(Igeom2, 0)

### 2.5 Harmonic mean filter

The filter is based on the expression:

$$I_{filtered}(x,y) = \dfrac{m \cdot n}{ \sum_{i=0}^m \sum_{j=0}^n \frac{1}{I(i,j)}}$$

The filter suppresses "*salt*" noise well and does not work with "*pepper*" noise.

Let's implement this filter.

At first, we need to create an image with borders, i.e., increase the image size not to think how to work with border items. As result image pixels will be shifted by the half of kernel, plus this gap will be filled with image border pixels. This is done by calling the `cv2.copyMakeBorder(I, ...)` function. It takes the following parameters: image, four gaps (top, bottom, left, right) and border type. We will use `cv2.BORDER_REPLICATE` to fill these gaps with border pixel values.

Next, we will use numpy matrix operations to calculate whole sums for the whole image at time. Don't forget that this filter works in floating point pixel values, so `uint8` images should be converted to floats and divided by $255$.

In [48]:
## Execute the harmonic mean filter
## @param[in] I An image to filter
## @param[in] kernel_size The size of the filter kernel
## @return Filtered image
def HarmonicMeanFilter(I, kernel_size):
  rows, cols = I.shape[0:2]

  # Filter parameters
  kernel = np.ones((kernel_size[0], kernel_size[1]))
  #kernel = kernel / kernel_size[0] / kernel_size[1]

  # Convert to float and make image with border
  if I.dtype == np.uint8:
    Icopy = I.astype(np.float32) / 255
  else:
    Icopy = I
  Icopy[Icopy == 0] = sys.float_info.epsilon
  Icopy = cv.copyMakeBorder(Icopy,
                            (kernel_size[0] - 1) // 2, kernel_size[0] // 2,
                            (kernel_size[1] - 1) // 2, kernel_size[1] // 2,
                            cv.BORDER_REPLICATE)

  # Do filter
  Iout = np.zeros(I.shape, np.float32)
  for i in range(kernel_size[0]):
    for j in range(kernel_size[1]):
      Iout = Iout + kernel[i, j] / Icopy[i:i + rows, j:j + cols]
  Iout = np.sum(kernel) / Iout

  # Convert back to uint if needed
  if (I.dtype == np.uint8):
    Iout = (255 * Iout).clip(0, 255).astype(np.uint8)

  return Iout

Now we can apply it to our noisy images.

In [ ]:
Ihmean = []
for img in Inoise:
  Ihmean.append((img[0], HarmonicMeanFilter(img[1], (5, 5))))
ShowImages(Ihmean, 0)

It can be seen that *pepper* noise got worse, but *salt* was cleaned. Let's try filtering *salt* only noise.

In [ ]:
Isalt = SaltAndPepper(I, 0.05, 1)
ShowImages([("Salt only", Isalt), ("Harmonic mean filter", HarmonicMeanFilter(Isalt, (5, 5)))])

In [ ]:
Ihmean2 = []
Isalt2 = SaltAndPepper(I2, 0.05, 1)
Ihmean2.append(('', HarmonicMeanFilter(Isalt2, (5, 5))))
for img in Inoise2[1:]:
  Ihmean2.append(('', HarmonicMeanFilter(img[1], (5, 5))))
ShowImages(Ihmean2, 0)

### 2.5 Contra Harmonic (Inverse) Mean Filter

The filter is based on the expression:

$$I_{filtered}(x,y) = \dfrac{\sum_{i=0}^m \sum_{j=0}^n I(i,j)^{Q+1}}{\sum_{i=0}^m \sum_{j=0}^n I(i,j)^Q }$$

where $Q$ is the filter order. The counterharmonic filter is a generalization of the averaging filters and for $Q>0$ suppresses noises of the "*pepper*" type, and if $Q<0$ then noises of the "*salt*" type, however, it is not possible to remove white and black points at the same time. If $Q=0$ the filter turns into an arithmetic one, and if $Q=-1$ then to the harmonic one.

Implementation of the counterharmonic mean filter is very similar to a harmonic one. The only difference is that now we have to calculate top and bottom parts for the whole image separately, and then divide them.

---
> **Self-work**
>
> Finish the implementation of the counterharmonic mean filter.
---

In [59]:
## Execute the harmonic mean filter
## @param[in] I An image to filter
## @param[in] kernel_size The size of the filter kernel
## @param[in] Q Filter parameter
## @return Filtered image
def CounterharmonicMeanFilter(I, kernel_size, Q):
  rows, cols = I.shape[0:2]

  # Filter parameters
  kernel = np.ones((kernel_size[0], kernel_size[1]))
  kernel = kernel / kernel_size[0] / kernel_size[1]

  # Convert to float and make image with border
  if I.dtype == np.uint8:
    Icopy = I.astype(np.float32) / 255
  else:
    Icopy = I
  Icopy = cv.copyMakeBorder(Icopy,
                            (kernel_size[0] - 1) // 2, kernel_size[0] // 2,
                            (kernel_size[1] - 1) // 2, kernel_size[1] // 2,
                            cv.BORDER_REPLICATE)

  # Calculate temporary matrices for I ** Q and I ** (Q + 1)
  # Calculate numerator and denominator for each pixel
  # Divide numerator by denominator
  # Convert back to uint if needed
  # And return the result

  Iout = np.zeros_like(I)

  import sys
  Icopy[Icopy == 0] = sys.float_info.epsilon

  Iout_num = np.zeros((rows, cols), dtype=np.float32)
  Iout_den = np.zeros((rows, cols), dtype=np.float32)

  for i in range(kernel_size[0]):
      for j in range(kernel_size[1]):
          # Числитель: I^(Q+1)
          Iout_num += kernel[i, j] * np.power(Icopy[i : i + rows, j : j + cols], Q + 1)
          # Знаменатель: I^Q
          Iout_den += kernel[i, j] * np.power(Icopy[i : i + rows, j : j + cols], Q)

  Iout = Iout_num / (Iout_den + 1e-9)

  # Convert back to uint if needed
  if (I.dtype == np.uint8):
    Iout = (255 * Iout).clip(0, 255).astype(np.uint8)


  return Iout

When filter implementation is finished we can apply it to our noisy images.

In [ ]:
Ichmean = []
for img in Inoise:
  Ichmean.append((img[0], CounterharmonicMeanFilter(img[1], (5, 5), 2)))
ShowImages(Ichmean, 0)

---
> ***Self-work***
>
> Try different $Q$ parameter values and tell which works better with different noise types.
---

In [ ]:
for q in [-0.5, 0.1, 0.5, 1, 2, 5]:
  Ichmean = []
  print(f"Parametr Q = {q}")
  for img in Inoise2:
    Ichmean.append(('', CounterharmonicMeanFilter(img[1], (5, 5), q)))
  ShowImages(Ichmean, 0)

### 2.6 Gaussian filter

The pixels in the sliding window that are closer to the analyzed pixel should have a greater influence on the filtering result than the extreme ones. Therefore, the mask weight coefficients can be described by the bell-shaped Gaussian function. During images filtering, a two-dimensional Gaussian filter is used:

$$G_{\sigma}=\dfrac{1}{2\pi\sigma^2}e^{-\frac{x^2+y^2}{2\sigma^2}}=\dfrac{1}{\sigma\sqrt{2\pi}}e^{\frac{-x^2}{2\sigma^2}}\cdot\dfrac{1}{\sigma\sqrt{2\pi}}e^{\frac{-y^2}{2\sigma^2}}$$

The larger the parameter $\sigma$, the more the image is blurred. Typically the filter radius $r=3\sigma$. In this case, the mask size $2r+1\times 2r+1$ and the matrix size is $6\sigma+1\times 6\sigma+1$. Outside this neighborhood, the values of the Gaussian function will be negligible.

OpenCV provides a function for Gaussian filter called `cv2.GaussianBlur(I, ksize, sigmaX, borderType)`. The additional parameters are used to specify the kernel size as a tuple (e.g., `ksize = (5, 5)` for $5 \times 5$ kernel), $\sigma$ value and border type (we will use `cv2.BORDER_REPLICATE`).

Let's run it.

---
> **Self-work**
>
> Run the Gaussian blur filter for noisy images. Use the $5 \times 5$ kernel and $\sigma$ value $1$. Which type of noise does this filter works better with?
---

In [ ]:
Igaussblur = []

for img in Inoise2:
  Igaussblur.append(('', cv.GaussianBlur(img[1], (5, 5), 1)))

ShowImages(Igaussblur, 0)

## Task 3. Nonlinear filtering

*Process the distorted images obtained in the first point with median, weighted median, rank and Wiener filtering for different sizes of the mask and its coefficients.*

*Optional: Implement adaptive median filtering.*

Low-pass filters are linear and are optimal when there is a normal distribution of noise in the digital image. Linear filters locally average impulse noise, smoothing images. To eliminate impulse noise, it is better to use non-linear filters, for example, *median* filters.

### 3.1 Median filter

The classical median filter uses a mask with unit coefficients. An arbitrary window shape can be set using zero coefficients. The pixel intensities in the window are represented as a column vector and sorted in ascending order. The filtered pixel is assigned the median (mean) intensity value in the series. The median element number after sorting can be calculated by the formula $n=\frac{N+1}{2}$, where $N$ is the number of pixels involved in sorting. When using OpenCV, the median filter can be executed by calling `cv2.medianBlur(I, ksize)` function.

| ![Median filter](images/median.png "Median Filter") |
|:--:|
| *Median filter* |


---
> ***Self-work***
>
> Run the median filter for noisy images. Which type of noise does this filter works better with?
---

In [ ]:
Imedian = []

for img in Inoise2:
  Imedian.append(('', cv2.medianBlur(img[1], 3)))

ShowImages(Imedian, 0)

### 3.2 Weighted median filter

In this median filtering modification in the mask, weights are used (numbers $2, 3$ etc.) to reflect more influence on the filtering result of pixels located closer to the element to be filtered. Each item is added the giver number of times to the array before sorting. Median filtering qualitatively removes impulse noise, and also does not introduce new intensity values in grayscale images. Increasing the size of the window increases the filter noise-canceling ability, but the objects outlines begin to distort. OpenCV does not profive weighted median filter, but it can be implemented.


In [98]:
## Weighted median filter
## @param[in] I An image to filter
## @param[in] weights Weights kernel
## @return The filtered image
def WightedMedianFilter(I, weights):
  rows, cols = I.shape[0:2]
  weights = weights.astype(np.uint16)
  kernel_size = weights.shape
  rank = int(weights.sum() // 2)

  # Convert to float and make image with border
  if I.dtype == np.uint8:
    Icopy = I.astype(np.float32) / 255
  else:
    Icopy = I#.astype(np.float32)
  Icopy = cv.copyMakeBorder(Icopy,
                            (kernel_size[0] - 1) // 2, kernel_size[0] // 2,
                            (kernel_size[1] - 1) // 2, kernel_size[1] // 2,
                            cv.BORDER_REPLICATE)
  # Fill arrays for each kernel item
  Ilayers = np.zeros(I.shape + (weights.sum(), ), dtype = np.float32)
  print("rank = {}".format(rank))
  if I.ndim == 2:
    n = 0
    for i in range(kernel_size[0]):
      for j in range(kernel_size[1]):
          for k in range(weights[i, j]):
            Ilayers[:, :, n] = Icopy[i:i + rows, j:j + cols]
            n += 1
  else:
    n = 0
    for i in range(kernel_size[0]):
      for j in range(kernel_size[1]):
        for k in range(weights[i, j]):
          Ilayers[:, :, :, n] = Icopy[i:i + rows, j:j + cols, :]
          n += 1

  # Sort
  Ilayers.sort()

  # And select one with rank
  if I.ndim == 2:
    Iout = Ilayers[:, :, rank]
  else:
    Iout = Ilayers[:, :, :, rank]

  # Convert back to uint if needed
  if (I.dtype == np.uint8):
    Iout = (255 * Iout).clip(0, 255).astype(np.uint8)

  return Iout

Let's run this filter doubling the weight of the central point.

In [ ]:
weights = np.ones((3, 3))
weights[1, 1] = 2
Iwmedian = []
for img in Inoise:
  Iwmedian.append((img[0], WightedMedianFilter(img[1], weights)))
ShowImages(Iwmedian, 0)

In [ ]:
weights = np.ones((3, 3))
weights[1, 1] = 2
Iwmedian2 = []
for img in Inoise2:
  Iwmedian2.append(('', WightedMedianFilter(img[1], weights)))
ShowImages(Iwmedian2, 0)

### 3.3 Rank filter

A median filtering generalization is a *rank filter* of order $r$ which selects a pixel with the given number from the resulting column vector of mask elements $r\in[1,N]$, which will be the result of filtering.

- If the number of pixels in the window $N$ is odd and $r=\frac{N+1}{2}$, then the rank filter is *median*.
- If $r=1$, the filter selects the lowest intensity value and is called *$min$-filter*.
- If $r=N$, the filter selects the maximum intensity value and is called *$max$-filter*.

Sometimes rank is written as a percentage, for example, for *$min$-filter* rank is $0%$, median filter is $50%$, *$max$-filter* is $100%$.


It can be noted that when implementing the weighted median filter we already had the `rank` variable which was used to find the median in the varying length array. We can make it a filter parameter to make it the weighted rank filter.

In [103]:
## Rank filter
## @param[in] I An image to filter
## @param[in] weights Weights kernel
## @param[in] rank The rank to use
## @return The filtered image
def WeightedRankFilter(I, weights, rank):
  rows, cols = I.shape[0:2]
  weights = weights.astype(np.uint16)
  kernel_size = weights.shape

  # Convert to float and make image with border
  if I.dtype == np.uint8:
    Icopy = I.astype(np.float32) / 255
  else:
    Icopy = I
  Icopy = cv.copyMakeBorder(Icopy,
                            (kernel_size[0] - 1) // 2, kernel_size[0] // 2,
                            (kernel_size[1] - 1) // 2, kernel_size[1] // 2,
                            cv.BORDER_REPLICATE)

  # Fill arrays for each kernel item
  Ilayers = np.zeros(I.shape + (weights.sum(), ), dtype = np.float32)
  if I.ndim == 2:
    n = 0
    for i in range(kernel_size[0]):
      for j in range(kernel_size[1]):
          for k in range(weights[i, j]):
            Ilayers[:, :, n] = Icopy[i:i + rows, j:j + cols]
            n += 1
  else:
    n = 0
    for i in range(kernel_size[0]):
      for j in range(kernel_size[1]):
        for k in range(weights[i, j]):
          Ilayers[:, :, :, n] = Icopy[i:i + rows, j:j + cols, :]
          n += 1

  # Sort
  Ilayers.sort()

  # And select one with rank
  if I.ndim == 2:
    Iout = Ilayers[:, :, rank]
  else:
    Iout = Ilayers[:, :, :, rank]

  # Convert back to uint if needed
  if (I.dtype == np.uint8):
    Iout = (255 * Iout).clip(0, 255).astype(np.uint8)

  return Iout

Of course a more simple and fast modification can be used to get only a rank filter.

In [104]:
## Rank filter
## @param[in] I An image to filter
## @param[in] kernel_size The filter kernel size
## @param[in] rank The rank to use
## @return The filtered image
def RankFilter(I, kernel_size, rank):
  rows, cols = I.shape[0:2]

  # Convert to float and make image with border
  if I.dtype == np.uint8:
    Icopy = I.astype(np.float32) / 255
  else:
    Icopy = I
  Icopy = cv.copyMakeBorder(Icopy,
                            (kernel_size[0] - 1) // 2, kernel_size[0] // 2,
                            (kernel_size[1] - 1) // 2, kernel_size[1] // 2,
                            cv.BORDER_REPLICATE)

  # Fill arrays for each kernel item
  Ilayers = np.zeros(I.shape + (kernel_size[0] * kernel_size[1], ), dtype = np.float32)
  if I.ndim == 2:
    for i in range(kernel_size[0]):
      for j in range(kernel_size[1]):
          Ilayers[:, :, i * kernel_size[1] + j] = Icopy[i:i + rows, j:j + cols]
  else:
    for i in range(kernel_size[0]):
      for j in range(kernel_size[1]):
        Ilayers[:, :, :, i * kernel_size[1] + j] = Icopy[i:i + rows, j:j + cols, :]

  # Sort
  Ilayers.sort()

  # And select one with rank
  if I.ndim == 2:
    Iout = Ilayers[:, :, rank]
  else:
    Iout = Ilayers[:, :, :, rank]

  # Convert back to uint if needed
  if (I.dtype == np.uint8):
    Iout = (255 * Iout).clip(0, 255).astype(np.uint8)

  return Iout

Let's run this filter and check the results.

---
> ***Self-work***
>
> Run the rank filter for $min$ and $max$ parameter values with all noise types.
---

Run the $min$ rank filter.

In [ ]:
Irank_min = []
ksize = (3, 3)

for img in Inoise2:
  filtered_min = RankFilter(img[1], ksize, rank=0)
  Irank_min.append(('', filtered_min))

ShowImages(Irank_min, 0)

Run the $max$ rank filter.

In [ ]:
Irank_max = []

for img in Inoise2:
    max_rank = ksize[0]*ksize[1] - 1
    filtered_max = RankFilter(img[1], ksize, rank=max_rank)
    Irank_max.append(('', filtered_max))

ShowImages(Irank_max, 0)

### 3.4 Adaptive median filter

In this filter modification, a sliding window of size $s\times s$ adaptively increases depending on the filtering result. Let us denote by $z_{min}$, $z_{max}$, $z_{med}$ minimum, maximum and median values of intensities in the window, $z_{i,j}$ is the pixel intensity value with coordinates $(i,j)$, $s_{max}$ is the maximum allowable window size. The adaptive median filtering algorithm consists of the following steps:

1. Calculating values $z_{min}$, $z_{max}$, $z_{med}$, $A_1=z_{med}-z_{min}$, $A_2=z_{med}-z_{max}$ of the pixel $(i,j)$ in the given window.
	1. If $A_1>0$ and $A_2<0$, go to step 2. Otherwise, increase the window size.
	2. If the current window size is $s\leqslant s_{max}$, repeat step 1.
	3. Otherwise, the filtering result is $z_{i,j}$.
2. Calculating values $B_1=z_{i,j}-z_{min}$, $B_2=z_{i,j}-z_{max}$.
	1. If $B_1>0$ and $B_2<0$, the filtering result is $z_{i,j}$.
	2. Otherwise, the filtering result is $z_{med}$.
3. Change coordinates $(i,j)$.
	1. If the image limit is not reached, go to step 1.
	2. Otherwise, the filtering is over.

The main idea is to increase the window size until the algorithm finds a median value that is not impulse noise, or until it reaches the maximum window size. In the latter case, the algorithm will return the value $z_{i,j}$.


---
> ***Self-work*** *(Optional)* (1 point)
>
> *Implement the adaptive median filter and run it with different noise types.*
---

In [110]:
## Adaptive median filter
## @param[in] I An image to filter
## @param[in] max_kernel_size The maximum filter kernel size
## @return The filtered image
def AdaptiveMedianFilter(I, max_kernel_size):
  need_rescale = False
  if I.dtype == np.uint8:
      Icopy = I.astype(np.float32) / 255.0
      need_rescale = True
  else:
      Icopy = I.copy()

  rows, cols = Icopy.shape[:2]
  Iout = np.zeros_like(Icopy)

  pad = max_kernel_size // 2
  Ipad = cv.copyMakeBorder(
      Icopy, pad, pad, pad, pad,
      borderType=cv.BORDER_REFLECT
  )

  for r in range(rows):
      for c in range(cols):
          s = 3
          pixel_value = Ipad[r + pad, c + pad]

          while True:
              half = s // 2
              region = Ipad[r + pad - half : r + pad + half + 1,
                            c + pad - half : c + pad + half + 1]

              Zmin = region.min()
              Zmax = region.max()
              Zmed = np.median(region)

              A1 = Zmed - Zmin
              A2 = Zmed - Zmax

              if A1 > 0 and A2 < 0:
                  B1 = pixel_value - Zmin
                  B2 = pixel_value - Zmax
                  if B1 > 0 and B2 < 0:
                      Iout[r, c] = pixel_value
                  else:
                      Iout[r, c] = Zmed
                  break
              else:
                  s += 2
                  if s > max_kernel_size:
                      Iout[r, c] = Zmed
                      break

  if need_rescale:
      Iout = (Iout * 255.0).clip(0, 255).astype(np.uint8)

  return Iout


Now run the adaptive median filter for all noise types. Compare the results.

In [ ]:
Iadaptive_med = []
for img in Inoise2:
  Iadaptive_med.append(('', AdaptiveMedianFilter(img[1], 9)))
ShowImages(Iadaptive_med, 0)

### 3.5 Wiener filter

Weiner filter uses Wiener's pixel-adaptive method based on statistics estimated from the local neighborhood of the each pixel.

$$\mu=\dfrac{1}{n\cdot m}  \sum_{i=0}^{m}\sum_{j=0}^{n} I(i,j)$$

$$\sigma^2=\dfrac{1}{n\cdot m}  \sum_{i=0}^{m}\sum_{j=0}^{n} I^2(i,j)-\mu^2$$

$$I_{new}(x,y)=\mu+\dfrac{\sigma^2-v^2}{\sigma^2}\left(I(x,y)-\mu\right)$$

Where $\mu$ is the average in the neighborhood, $\sigma^2$ is the variance, and $v^2$ is the noise variance.

OpenCV does not provide an implementation for this filter, but it can be easily implemented.

In [114]:
## The Weiner filter
## @param[in] img_noisy An image to filter
## @param[in] kernel Filter kernel
def WeinerFilter(I, kernel):
  rows, cols = I.shape[0:2]

  # Convert to float32
  if I.dtype == np.uint8:
    Iout = I.astype(np.float32) / 255
  else:
    Iout = np.copy(I)

  kernel_size = kernel.shape

  # Convert to float and make image with border
  if I.dtype == np.uint8:
    Icopy = I.astype(np.float32) / 255
  else:
    Icopy = I
  Icopy = cv.copyMakeBorder(Icopy,
                            (kernel_size[0] - 1) // 2, kernel_size[0] // 2,
                            (kernel_size[1] - 1) // 2, kernel_size[1] // 2,
                            cv.BORDER_REPLICATE)

  # Split into layers
  bgr_planes = cv.split(Icopy)
  bgr_planes_2 = []

  kernel_power = np.power(kernel, 2)

  # For all layers
  for plane in bgr_planes:
    # Calculate temporary matrices for I ** 2
    plane_power = np.power(plane, 2)

    m = np.zeros(I.shape[0:2], np.float32)
    q = np.zeros(I.shape[0:2], np.float32)

    # Calculate variance values
    for i in range(kernel_size[0]):
      for j in range(kernel_size[1]):
        m = m + kernel[i, j] * plane[i:i + rows, j:j + cols]
        q = q + kernel_power[i, j] * plane_power[i:i + rows, j:j + cols]
    m = m / np.sum(kernel)
    q = q / np.sum(kernel)
    q = q - m * m

    # Calculate noise as an average variance
    v = np.sum(q) / I.size

    # Do filter
    plane_2 = plane[(kernel_size[0] - 1) // 2:(kernel_size[0] - 1) // 2 + rows,
      (kernel_size[1] - 1) // 2:(kernel_size[1] - 1) // 2 + cols]
    plane_2 = np.where(q < v, m, (plane_2 - m) * (1 - v / q) + m)
    bgr_planes_2.append(plane_2)

  # Merge image back
  Iout = cv.merge(bgr_planes_2)

  # Convert back to uint if needed
  if (I.dtype == np.uint8):
    Iout = (255 * Iout).clip(0, 255).astype(np.uint8)

  return Iout

---
> ***Self-work***
>
> Run the Weiner filter with all of our noisy images. Use the $5 \times 5$ kernel filled with $1$.
---

In [ ]:
Iweiner = []
ker = np.ones((5, 5))
for img in Inoise2:
  Iweiner.append(('', WeinerFilter(img[1], kernel)))
ShowImages(Iweiner, 0)

## Task 4. High-pass filtering

*Select source image. Detect edges with Roberts, Prewitt, Sobel, Laplace filters, Canny algorithm.*

High-pass spatial filters enhance high-pass components (areas of strong intensity variation) and attenuate low-pass components of the image. They are used to highlight intensity differences and define edges (contours) in images. As a result of applying high-pass filters, the image is sharpened.

| ![High-pass filtering](images/high-pass.png "High-pass Filter") |
|:--:|
| *Intensity function and its first derivative, the maximum of the derivative corresponds to the edge* |

High-pass filters approximate the computation of directional derivatives, while the increment of the argument $\Delta x$ is taken equal to 1 or 2. The main distinctive features are:

- filter mask coefficients can take negative values;
- the sum of all coefficients is zero.

### 4.1 Roberts filter

The Roberts filter works with the minimum dimensionality mask allowed for the derivative calculation $2 \times 2$, therefore it is fast and quite efficient. Possible options for masks for finding the gradient along the axes $Ox$ and $Oy$:

$$
	G_x =
	\begin{bmatrix}
		1 & -1 \\
		0 & 0
	\end{bmatrix},
	G_y =
	\begin{bmatrix}
		1 & 0 \\
		-1 & 0
	\end{bmatrix},
$$

or

$$
	G_x =
	\begin{bmatrix}
		1 & 0 \\
		0 & -1
	\end{bmatrix},
	G_y =
	\begin{bmatrix}
		0 & 1 \\
		-1 & 0
	\end{bmatrix}.
$$

As a result of applying the Roberts differential operator, we obtain an estimate of the gradient in the directions $G_x$ and $G_y$. The all edge detectors gradient modulus can be calculated by the formula $G=\sqrt{G_x^2+G_y^2}=|G_x|+|G_y|$, and gradient direction by the formula $\arctan{ \left(\dfrac{G_y}{G_x}\right) }$.



In OpenCV it is implemented by calling `cv2.filter2D()` function with two matrices $G_x$ and $G_y$ and then calculating their square mean by `cv2.magnitude()` function.

In [122]:
## The Roberts filter
## @param[in] I An image to filter
## @return The filtered image
def RobertsFilter(I):
  # Convert to float
  if I.dtype == np.uint8:
    Iout = I.astype(np.float32) / 255
  else:
    Iout = np.copy(I)

  # Define kernels for X and Y
  kernel_x = np.array([[1, -1], [ 0, 0]])
  kernel_y = np.array([[1,  0], [-1, 0]])

  # Perform convolution
  Ix = cv.filter2D(Iout, -1, kernel_x)
  Iy = cv.filter2D(Iout, -1, kernel_y)
  Iout = cv.magnitude(Ix, Iy)

  # Convert back to uint if needed
  if I.dtype == np.uint8:
    Iout = (255 * Iout).clip(0, 255).astype(np.uint8)

  return Iout

As this filter is not used to filter noise, so we will run it for our initial image and check what borders are found.

In [ ]:
Iroberts = RobertsFilter(I)
ShowImages([("Source image", I),
            ("Roberts filter", Iroberts)], 2)

In [ ]:
Iroberts2 = RobertsFilter(I2)
ShowImages([("Source image", I2),
            ('', Iroberts2)], 0)

### 4.2 Prewitt Filter

This approach uses two orthogonal masks of size $3\times 3$, allowing you to more accurately calculate the derivatives along the axes $Ox$ and $Oy$:

$$
	G_x =
	\begin{bmatrix}
		-1 & 0 & 1 \\
		-1 & 0 & 1 \\
		-1 & 0 & 1
	\end{bmatrix},
	G_y =
	\begin{bmatrix}
		-1 & -1 & -1 \\
		0 & 0 & 0 \\
		1 & 1 & 1
	\end{bmatrix}.
$$

Implementation is very similar to implementation of the Roberts filter.

---
> ***Self-work***
>
> Implement the Prewitt filter and run it with the source image.
---

In [118]:
## The Prewitt filter
## @param[in] I An image to filter
## @return The filtered image
def PrewittFilter(I):
  # Convert to float
  # Define kernels for X and Y
  # Perform convolution
  # Convert back to uint if needed

  Iout = np.zeros_like(I)

  if I.dtype == np.uint8:
    Icopy = I.astype(np.float32) / 255.0
    need_rescale = True
  else:
    Icopy = I.copy()
    need_rescale = False

  Px = np.array([[-1,  0,  1],
                  [-1,  0,  1],
                  [-1,  0,  1]], dtype=np.float32)

  Py = np.array([[-1, -1, -1],
                  [ 0,  0,  0],
                  [ 1,  1,  1]], dtype=np.float32)

  Gx = cv.filter2D(Icopy, ddepth=-1, kernel=Px, borderType=cv.BORDER_REPLICATE)
  Gy = cv.filter2D(Icopy, ddepth=-1, kernel=Py, borderType=cv.BORDER_REPLICATE)

  G = np.sqrt(Gx**2 + Gy**2)

  if need_rescale:
      Iout = (G * 255.0).clip(0, 255).astype(np.uint8)
  else:
      Iout = G

  return Iout

Now run it and check results.

In [ ]:
Iprewitt = PrewittFilter(I)
ShowImages([("Source image", I),
            ("Prewitt filter", Iprewitt)], 2)

In [ ]:
Iprewitt2 = PrewittFilter(I2)
ShowImages([("Source image", I2),
            ('', Iprewitt2)], 0)

### 4.3 Sobel filter

This approach is similar to the Roberts filter, but different mask weights are used. A typical example of a Sobel filter:

$$
	G_x =
	\begin{bmatrix}
		-1 & 0 & 1 \\
		-2 & 0 & 2 \\
		-1 & 0 & 1
	\end{bmatrix},
	G_y =
	\begin{bmatrix}
		1 & 2 & 1 \\
		0 & 0 & 0 \\
		-1 & -2 & -1
	\end{bmatrix}.
$$

This filter can be implemented manually as was described above or can use `cv2.Sobel(I, ddepth, dx, dy)` OpenCV function.

---
> ***Self-work***
>
> Implement the Sobel filter **without** using OpenCV `cv2.Sobel()` function. Run this filter with the source image.
---

In [126]:
## The Sobel filter
## @param[in] I An image to filter
## @return The filtered image
def SobelFilter(I):
  # Convert to float
  # Define kernels for X and Y
  # Perform convolution
  # Convert back to uint if needed

  Iout = np.zeros_like(I)

  if I.dtype == np.uint8:
    Icopy = I.astype(np.float32) / 255.0
    need_rescale = True
  else:
    Icopy = I.copy()
    need_rescale = False

  Sx = np.array([[-1,  0,  1],
                  [-2,  0,  2],
                  [-1,  0,  1]], dtype=np.float32)
  Sy = np.array([[-1, -2, -1],
                  [ 0,  0,  0],
                  [ 1,  2,  1]], dtype=np.float32)

  Gx = cv.filter2D(Icopy, ddepth=-1, kernel=Sx, borderType=cv.BORDER_REPLICATE)
  Gy = cv.filter2D(Icopy, ddepth=-1, kernel=Sy, borderType=cv.BORDER_REPLICATE)

  G = np.sqrt(Gx**2 + Gy**2)

  if need_rescale:
      Iout = (G * 255.0).clip(0, 255).astype(np.uint8)
  else:
      Iout = G

  return Iout

  return Iout

Now run it and check results.

In [ ]:
Isobel = SobelFilter(I)
ShowImages([("Source image", I), ("Sobel filter", Isobel)], 2)

In [ ]:
Isobel2 = SobelFilter(I2)
ShowImages([("Source image", I2), ('', Isobel2)], 0)

### 4.4 Laplace filter

Laplace filter uses approximation of the second derivatives along the axes $Ox$ and $Oy$ as opposed to previous approaches using the first derivative:

| ![Laplace filter](images/laplace.png "Laplace filter") |
|:--:|
| *The second derivative of the brightness function changes sign (passes through zero at the point corresponding to the edge* |

$$L(I(x,y)) = \frac{\partial^2 I}{\partial x^2}+\frac{\partial^2 I}{\partial y^2}$$

The above formula can be approximated by the following mask:

$$
	w =
	\begin{bmatrix}
		0 & -1 & 0 \\
		-1 & 4 & -1 \\
		0 & -1 & 0
	\end{bmatrix}
$$

Let's implement this filter according to a given forumla. It should be noted that this filter gives us the gradient, in other words, it can result in the negative values. For display purposes we will use the absolute value of the Laplace filter result (by applying `np.abs()` to the floating point value result before converting back to `uint8`). However in real cases this filter result should be used as is in floating point values for the further image processing steps.

There is the `cv2.Laplacian()` OpenCV function, however it **doesn't implement the Laplace filter**, *instead it uses the variation of the Sobel filter* along two axis with calculation of their magnitude. So it can't be used here.

---
> **Self-work**
>
> Implement the Laplace filter **without** using OpenCV `cv2.Laplacian()` function. Don't forget to apply `np.abs()` to the result before converting it to $[0, 255]$ range. Run the Laplace filter implementation with the source image.
---

In [130]:
def LaplaceFilter(I):
  # Convert to float
  # Define kernel
  # Perform convolution
  # Convert back to uint if needed

  if I.dtype == np.uint8:
    Icopy = I.astype(np.float32) / 255.0
    need_rescale = True
  else:
    Icopy = I.copy()
    need_rescale = False

  laplace_kernel = np.array([
      [ 0,  1,  0],
      [ 1, -4,  1],
      [ 0,  1,  0]
  ], dtype=np.float32)

  Ifiltered = cv.filter2D(Icopy, ddepth=-1, kernel=laplace_kernel, borderType=cv.BORDER_REPLICATE)

  Iabs = np.abs(Ifiltered)

  if need_rescale:
      Iout = (Iabs * 255.0).clip(0, 255).astype(np.uint8)
  else:
      Iout = Iabs

  return Iout

Now let's run it.

In [ ]:
Ilaplace = LaplaceFilter(I)
ShowImages([("Source image", I), ("Laplace filter", Ilaplace)], 2)

In [ ]:
Ilaplace2 = LaplaceFilter(I2)
ShowImages([("Source image", I2), ('', Ilaplace2)], 0)

### 4.5 Canny algorithm

One of the most widespread and efficient algorithms for extracting edges in an image is *Canny algorithm*. This algorithm allows not only to define edge pixels, but also connected boundary lines. The algorithm consists of the following steps:

1. Elimination of small details by smoothing the original image using a Gaussian filter.
2. Using the Sobel differential operator to determine the values of the all image pixels gradient modulus, and the calculation result is rounded by steps $45^{\circ}$.
3. Analysis of the gradient modules values of the pixels located orthogonally to the investigated one. If the gradient modulus value of the investigated pixel is greater than orthogonal neighboring pixels, then it is *edge*, otherwise it is *non-maximum*.
4. Performing double threshold filtering of edge pixels selected in the previous step:
 - If the gradient modulus value greater the threshold $t_2$, then the edge presence in a pixel is valid.
 - If the gradient modulus value lower the threshold $t_1$, then the pixel is definitely not edge.
 - If the gradient modulus value in a range of $[t_1, t_2]$, then such a pixel is considered as $\textit {ambiguous}$.
5. Suppresses all ambiguous pixels not associated with valid pixels by 8-connectivity.

In OpenCV the Canny algorithm can be executed using `cv2.Canny(I, t1, t2)` function. `t1` and `t2` parameters define thresholding step thresholds.

In [ ]:
Icanny = cv.Canny(I, 50, 220)
ShowImages([("Source image", I),
            ("Canny algorithm", Icanny)], 2)

---
> **Self-work**
>
> Run the Canny algorithm for your image.
---

In [ ]:
Icanny2 = cv.Canny(I2, 50, 220)
ShowImages([("Source image", I2),
            ('', Icanny2)], 0)

### 4.6 High-pass filters summary

Let's check all high-pass filter results side-by-side.

In [ ]:
ShowImages([("Source image", I),
            ("Roberts filter", Iroberts),
            ("Prewitt filter", Iprewitt),
            ("Sobel filter", Isobel),
            ("Laplace filter", Ilaplace),
            ("Canny algorithm", Icanny)], 3)

---
> ***Self-work***
>
> *Answer questions and concdlude your work results.*
---

 ## Questions

Please answer the following questions:

 - What are the main disadvantages of adaptive image filtering methods?
 > Put your answer here

 - Which value of the $Q$ parameter will make the counterharmonic filter to work work as an arithmetic mean filter, and which value will make it a harmonic one?
 > Put your answer here

 - What operators can be used to detect edges in the image?
 > Put your answer here

 - Why, as a rule, is low-pass filtering performed at the first step of edge detection?
 > Put your answer here

## Conclusion

What have you learned with this task? Don't forget to conclude it.
 > Put your conclusion here
